### Install Things

In [1]:
!pip install pandasql --quiet

### Import Things

In [2]:
import numpy as np
import pandas as pd
import pandasql as psql
import warnings
import math
from pandas.errors import SettingWithCopyWarning

pd.set_option("display.max_columns", None)
warnings.simplefilter(action = "ignore", category = SettingWithCopyWarning)

### Some Functions

In [3]:
# Keep only shots and goals function
def keep_SOG(df):
    df = df[(df["typeDescKey"] == "shot-on-goal") | (df["typeDescKey"] == "goal")]

    return(df)

In [4]:
# Fix pbp data types
def fix_pbp_types(pbp_df):
    pbp_df["situationCode"] = pbp_df["situationCode"].astype(int).astype(str).str.pad(width = 4, side = "left", fillchar = "0")
    pbp_df["gameId"] = pbp_df["gameId"].astype(str)
    pbp_df["periodDescriptor.number"] = pbp_df["periodDescriptor.number"].astype(int)
    pbp_df["details.eventOwnerTeamId"] = pbp_df["details.eventOwnerTeamId"].astype(int).astype(str)

    return(pbp_df)

In [5]:
# Fix schedule data types
def fix_schedule_types(schedule_df):
    schedule_df["id"] = schedule_df["id"].astype(str)
    schedule_df["awayTeam.id"] = schedule_df["awayTeam.id"].astype(str)
    schedule_df["homeTeam.id"] = schedule_df["homeTeam.id"].astype(str)

    return(schedule_df)

In [6]:
# Determine which direction the shot is being taken
def shot_side(row):
    if row["homeTeamShot"] == True:
        if row["homeTeamDefendingSide"] == "left":
            return "right"
        else:
            return "left"
    else:
        return row["homeTeamDefendingSide"]

In [7]:
# Determine shot distance
def get_shot_distance(row):
    return math.hypot(89 - row["adj.xCoord"], 0 - row["adj.yCoord"])

In [8]:
# Determine shot distance class
def get_shot_distance_class(df, model):
    # Create shot distance buckets
    if model == "krzy05":
        conditions = [
            df["shotDistance"] < 10,
            (df["shotDistance"] >= 10) & (df["shotDistance"] < 13),
            (df["shotDistance"] >= 13) & (df["shotDistance"] < 15),
            (df["shotDistance"] >= 15) & (df["shotDistance"] < 17),
            (df["shotDistance"] >= 17) & (df["shotDistance"] < 23),
            (df["shotDistance"] >= 23) & (df["shotDistance"] < 32),
            (df["shotDistance"] >= 32) & (df["shotDistance"] < 37),
            (df["shotDistance"] >= 37) & (df["shotDistance"] < 39),
            (df["shotDistance"] >= 39) & (df["shotDistance"] < 45),
            (df["shotDistance"] >= 45) & (df["shotDistance"] < 58),
            df["shotDistance"] >= 58
        ]

        choices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

        df["shotDistanceClass"] = np.select(conditions, choices, default = None)
        return(df)

In [9]:
# Get Strength at time of event
def get_strength(row):
    if row["homeTeamShot"] == True:
        if row["home_players"] > row["away_players"]:
            return "PP"
        elif row["home_players"] < row["away_players"]:
            return "SH"
        else:
            return "EV"
    elif row["homeTeamShot"] == False:
        if row["home_players"] > row["away_players"]:
            return "SH"
        elif row["home_players"] < row["away_players"]:
            return "PP"
        else:
            return "EV"

### Get Data

In [10]:
skaters = pd.read_csv("skaters.csv")
goalies = pd.read_csv("goalies.csv")
pbp = pd.read_csv("pbp.csv")
schedule = pd.read_csv("schedule.csv")
pbp.head()

/var/folders/5k/vm4wbtfd3hz2hq37wm4d1_8h0000gp/T/ipykernel_90426/863690396.py:3: DtypeWarning: Columns (45,46,47) have mixed types. Specify dtype option on import or set low_memory=False.
  pbp = pd.read_csv("pbp.csv")


,eventId,timeInPeriod,timeRemaining,situationCode,homeTeamDefendingSide,typeCode,typeDescKey,sortOrder,periodDescriptor.number,periodDescriptor.periodType,periodDescriptor.maxRegulationPeriods,details.eventOwnerTeamId,details.losingPlayerId,details.winningPlayerId,details.xCoord,details.yCoord,details.zoneCode,details.hittingPlayerId,details.hitteePlayerId,details.shotType,details.shootingPlayerId,details.goalieInNetId,details.awaySOG,details.homeSOG,details.reason,details.typeCode,details.descKey,details.duration,details.committedByPlayerId,details.drawnByPlayerId,details.blockingPlayerId,details.playerId,details.secondaryReason,details.scoringPlayerId,details.scoringPlayerTotal,details.assist1PlayerId,details.assist1PlayerTotal,details.assist2PlayerId,details.assist2PlayerTotal,details.awayScore,details.homeScore,details.discreteClip,details.servedByPlayerId,gameId,periodDescriptor.otPeriods,pptReplayUrl,details.highlightClipSharingUrl,details.highlightClipSharingUrlFr,details.highlightClip,details.highlightClipFr,details.discreteClipFr
0,51.0,00:00,20:00,1551.0,right,520.0,period-start,8.0,1.0,REG,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021010001,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,53.0,00:00,20:00,1551.0,right,502.0,faceoff,9.0,1.0,REG,3.0,10.0,8480018.0,8475166.0,0.0,0.0,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021010001,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,101.0,00:15,19:45,1551.0,right,503.0,hit,10.0,1.0,REG,3.0,10.0,NaN,NaN,-96.0,26.0,O,8475166.0,8481014.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021010001,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,54.0,00:17,19:43,1551.0,right,506.0,shot-on-goal,11.0,1.0,REG,3.0,10.0,NaN,NaN,-49.0,7.0,O,NaN,NaN,snap,8480043.0,8480051.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021010001,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,55.0,00:27,19:33,1551.0,right,507.0,missed-shot,12.0,1.0,REG,3.0,10.0,NaN,NaN,-31.0,39.0,O,NaN,NaN,wrist,8480043.0,8480051.0,NaN,NaN,wide-of-net,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021010001,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# Fix universal dataset data types
schedule = fix_schedule_types(schedule)

### Split Play-By-Play Into Train (2021-22, 2022-23, 2023-24) and Test (2024-25), Additionally Filter for Regular Season Only

In [12]:
# Set "gameId" to a string
pbp["gameId"] = pbp["gameId"].apply(str)

# Split into training and testing events
train_pbp = pbp.loc[pbp.gameId.str.startswith(("2021", "2022", "2023"), na=False)]
test_pbp = pbp.loc[pbp.gameId.str.startswith("2024", na=False)]
print("Training events: ", len(train_pbp))
print("Testing events: ", len(test_pbp))

Training events:  1404747
Testing events:  479352


In [13]:
# Keep only regular season games (remove pre-season and playoffs)
train_pbp = train_pbp.loc[train_pbp["gameId"].str[5] == "2"]
test_pbp = test_pbp.loc[test_pbp["gameId"].str[5] == "2"]
print("Training events: ", len(train_pbp))
print("Testing events: ", len(test_pbp))

Training events:  1227064
Testing events:  420073


In [14]:
# Create timeInPeriodSeconds
train_pbp["timeInPeriodSeconds"] = pd.to_timedelta("00:" + train_pbp["timeInPeriod"]).dt.total_seconds().astype(int)
test_pbp["timeInPeriodSeconds"] = pd.to_timedelta("00:" + test_pbp["timeInPeriod"]).dt.total_seconds().astype(int)

In [15]:
# Create a two new two new columns to help define rebounds (lastEvent & timeSinceLastEvent)
train_pbp["lastEvent"] = train_pbp.groupby("gameId")["typeDescKey"].shift(1)
test_pbp["lastEvent"] = test_pbp.groupby("gameId")["typeDescKey"].shift(1)

train_pbp["timeInPeriodShifted"] = train_pbp.groupby("gameId")["timeInPeriodSeconds"].shift(1)
test_pbp["timeInPeriodShifted"] = test_pbp.groupby("gameId")["timeInPeriodSeconds"].shift(1)

train_pbp["timeSinceLastEvent"] = train_pbp["timeInPeriodSeconds"] - train_pbp["timeInPeriodShifted"]
test_pbp["timeSinceLastEvent"] = test_pbp["timeInPeriodSeconds"] - test_pbp["timeInPeriodShifted"]

### [Krzywicki (2005)](https://www.hockeyanalytics.com/Research_files/Shot_Quality_Krzywicki.pdf)

In [16]:
# Variables
# - Distance (<10ft, 10ft-12ft, 13ft-14ft, 15ft-16ft, 17ft-22ft, 23ft-31ft, 32ft-36ft, 37ft-38ft
#             39ft-44ft, 45ft-57ft, >=58ft)
# - Shot Type (Wrap, Slap, Wrist, Snap, Backhand, Tip)
# - Rebound (Yes, No)
# - Situation (Even Strength, Shorthanded, Powerplay)

In [17]:
# Keep only shot and goal events
train_pbp_krzy05 = keep_SOG(train_pbp)
test_pbp_krzy05 = keep_SOG(test_pbp)
print("Training shots: ", len(train_pbp_krzy05))
print("Testing shots: ", len(test_pbp_krzy05))

Training shots:  246134
Testing shots:  74641


In [18]:
# Fix data types
train_pbp_krzy05 = fix_pbp_types(train_pbp_krzy05)
test_pbp_krzy05 = fix_pbp_types(test_pbp_krzy05)

In [19]:
# Reduce columns to those neccesary for the model
keep_pbp = ["timeInPeriod", "timeRemaining", "situationCode", "typeDescKey", "periodDescriptor.number", "details.eventOwnerTeamId", "details.xCoord", "details.yCoord", "details.shotType", "gameId", "homeTeamDefendingSide", "lastEvent", "timeSinceLastEvent"]
keep_schedule = ["id", "awayTeam.id", "homeTeam.id"]
train_pbp_krzy05 = pd.merge(train_pbp_krzy05[keep_pbp],schedule[keep_schedule], left_on = "gameId", right_on = "id", how = "inner")
test_pbp_krzy05 = pd.merge(test_pbp_krzy05[keep_pbp],schedule[keep_schedule], left_on = "gameId", right_on = "id", how = "inner")
train_pbp_krzy05.head()

,timeInPeriod,timeRemaining,situationCode,typeDescKey,periodDescriptor.number,details.eventOwnerTeamId,details.xCoord,details.yCoord,details.shotType,gameId,homeTeamDefendingSide,lastEvent,timeSinceLastEvent,id,awayTeam.id,homeTeam.id
0,00:15,19:45,1551,shot-on-goal,1,8,71.0,13.0,slap,2021020003,right,hit,4.0,2021020003,8,10
1,00:18,19:42,1551,shot-on-goal,1,8,78.0,-22.0,wrist,2021020003,right,missed-shot,0.0,2021020003,8,10
2,01:16,18:44,1551,shot-on-goal,1,10,-60.0,-18.0,wrist,2021020003,right,hit,26.0,2021020003,8,10
3,01:23,18:37,1551,shot-on-goal,1,8,64.0,-16.0,wrist,2021020003,right,shot-on-goal,7.0,2021020003,8,10
4,02:50,17:10,1551,shot-on-goal,1,8,31.0,20.0,slap,2021020003,right,blocked-shot,2.0,2021020003,8,10


In [20]:
# Drop rows with details.shotType (38 train, 24 test), this is the only column containing NaN
train_pbp_krzy05 = train_pbp_krzy05[train_pbp_krzy05["details.shotType"].notna()]
test_pbp_krzy05 = test_pbp_krzy05[test_pbp_krzy05["details.shotType"].notna()]
print("Training shots: ", len(train_pbp_krzy05))
print("Testing shots: ", len(test_pbp_krzy05))

Training shots:  246096
Testing shots:  74617


In [21]:
#Create an abs.xCoord column, so shots are in one direction
train_pbp_krzy05["abs.xCoord"] = train_pbp_krzy05["details.xCoord"].abs()
test_pbp_krzy05["abs.xCoord"] = test_pbp_krzy05["details.xCoord"].abs()

# Determine whether home team took the shot
train_pbp_krzy05["homeTeamShot"] = train_pbp_krzy05["details.eventOwnerTeamId"] == train_pbp_krzy05["homeTeam.id"]
test_pbp_krzy05["homeTeamShot"] = test_pbp_krzy05["details.eventOwnerTeamId"] == test_pbp_krzy05["homeTeam.id"]

# Determine which direction the shot was taken
train_pbp_krzy05["shotSide"] = train_pbp_krzy05.apply(shot_side, axis = 1)
test_pbp_krzy05["shotSide"] = train_pbp_krzy05.apply(shot_side, axis = 1)

# Adjust shots taken towards the left net
train_pbp_krzy05["adj.xCoord"] = train_pbp_krzy05.apply(lambda row: abs(row["details.xCoord"]) if row["shotSide"] == "left" else row["details.xCoord"], axis = 1)
train_pbp_krzy05["adj.yCoord"] = train_pbp_krzy05.apply(lambda row: abs(row["details.yCoord"]) if row["shotSide"] == "left" else row["details.yCoord"], axis = 1)

test_pbp_krzy05["adj.xCoord"] = test_pbp_krzy05.apply(lambda row: abs(row["details.xCoord"]) if row["shotSide"] == "left" else row["details.xCoord"], axis = 1)
test_pbp_krzy05["adj.yCoord"] = test_pbp_krzy05.apply(lambda row: abs(row["details.yCoord"]) if row["shotSide"] == "left" else row["details.yCoord"], axis = 1)

train_pbp_krzy05.head()

,timeInPeriod,timeRemaining,situationCode,typeDescKey,periodDescriptor.number,details.eventOwnerTeamId,details.xCoord,details.yCoord,details.shotType,gameId,homeTeamDefendingSide,lastEvent,timeSinceLastEvent,id,awayTeam.id,homeTeam.id,abs.xCoord,homeTeamShot,shotSide,adj.xCoord,adj.yCoord
0,00:15,19:45,1551,shot-on-goal,1,8,71.0,13.0,slap,2021020003,right,hit,4.0,2021020003,8,10,71.0,False,right,71.0,13.0
1,00:18,19:42,1551,shot-on-goal,1,8,78.0,-22.0,wrist,2021020003,right,missed-shot,0.0,2021020003,8,10,78.0,False,right,78.0,-22.0
2,01:16,18:44,1551,shot-on-goal,1,10,-60.0,-18.0,wrist,2021020003,right,hit,26.0,2021020003,8,10,60.0,True,left,60.0,18.0
3,01:23,18:37,1551,shot-on-goal,1,8,64.0,-16.0,wrist,2021020003,right,shot-on-goal,7.0,2021020003,8,10,64.0,False,right,64.0,-16.0
4,02:50,17:10,1551,shot-on-goal,1,8,31.0,20.0,slap,2021020003,right,blocked-shot,2.0,2021020003,8,10,31.0,False,right,31.0,20.0


In [22]:
# Create shotDistance column
train_pbp_krzy05["shotDistance"] = train_pbp_krzy05.apply(get_shot_distance, axis = 1)
test_pbp_krzy05["shotDistance"] = test_pbp_krzy05.apply(get_shot_distance, axis = 1) 

# Create shotDistanceClass column
train_pbp_krzy05 = get_shot_distance_class(train_pbp_krzy05, "krzy05")
test_pbp_krzy05 = get_shot_distance_class(test_pbp_krzy05, "krzy05")

train_pbp_krzy05.head()

,timeInPeriod,timeRemaining,situationCode,typeDescKey,periodDescriptor.number,details.eventOwnerTeamId,details.xCoord,details.yCoord,details.shotType,gameId,homeTeamDefendingSide,lastEvent,timeSinceLastEvent,id,awayTeam.id,homeTeam.id,abs.xCoord,homeTeamShot,shotSide,adj.xCoord,adj.yCoord,shotDistance,shotDistanceClass
0,00:15,19:45,1551,shot-on-goal,1,8,71.0,13.0,slap,2021020003,right,hit,4.0,2021020003,8,10,71.0,False,right,71.0,13.0,22.203603,4
1,00:18,19:42,1551,shot-on-goal,1,8,78.0,-22.0,wrist,2021020003,right,missed-shot,0.0,2021020003,8,10,78.0,False,right,78.0,-22.0,24.596748,5
2,01:16,18:44,1551,shot-on-goal,1,10,-60.0,-18.0,wrist,2021020003,right,hit,26.0,2021020003,8,10,60.0,True,left,60.0,18.0,34.132096,6
3,01:23,18:37,1551,shot-on-goal,1,8,64.0,-16.0,wrist,2021020003,right,shot-on-goal,7.0,2021020003,8,10,64.0,False,right,64.0,-16.0,29.681644,5
4,02:50,17:10,1551,shot-on-goal,1,8,31.0,20.0,slap,2021020003,right,blocked-shot,2.0,2021020003,8,10,31.0,False,right,31.0,20.0,61.351447,10


In [23]:
# Create rebound column as defined by Ryder (2004)
train_pbp_krzy05["Rebound"] = ((train_pbp_krzy05["lastEvent"] == "shot-on-goal") & (train_pbp_krzy05["timeSinceLastEvent"] >=0) & (train_pbp_krzy05["timeSinceLastEvent"] <= 2))
test_pbp_krzy05["Rebound"] = ((test_pbp_krzy05["lastEvent"] == "shot-on-goal") & (test_pbp_krzy05["timeSinceLastEvent"] >=0) & (test_pbp_krzy05["timeSinceLastEvent"] <= 2))
train_pbp_krzy05.head()

,timeInPeriod,timeRemaining,situationCode,typeDescKey,periodDescriptor.number,details.eventOwnerTeamId,details.xCoord,details.yCoord,details.shotType,gameId,homeTeamDefendingSide,lastEvent,timeSinceLastEvent,id,awayTeam.id,homeTeam.id,abs.xCoord,homeTeamShot,shotSide,adj.xCoord,adj.yCoord,shotDistance,shotDistanceClass,Rebound
0,00:15,19:45,1551,shot-on-goal,1,8,71.0,13.0,slap,2021020003,right,hit,4.0,2021020003,8,10,71.0,False,right,71.0,13.0,22.203603,4,False
1,00:18,19:42,1551,shot-on-goal,1,8,78.0,-22.0,wrist,2021020003,right,missed-shot,0.0,2021020003,8,10,78.0,False,right,78.0,-22.0,24.596748,5,False
2,01:16,18:44,1551,shot-on-goal,1,10,-60.0,-18.0,wrist,2021020003,right,hit,26.0,2021020003,8,10,60.0,True,left,60.0,18.0,34.132096,6,False
3,01:23,18:37,1551,shot-on-goal,1,8,64.0,-16.0,wrist,2021020003,right,shot-on-goal,7.0,2021020003,8,10,64.0,False,right,64.0,-16.0,29.681644,5,False
4,02:50,17:10,1551,shot-on-goal,1,8,31.0,20.0,slap,2021020003,right,blocked-shot,2.0,2021020003,8,10,31.0,False,right,31.0,20.0,61.351447,10,False


In [24]:
# Create a column to denote strength

# Breakup code 
train_pbp_krzy05["home_goalie"] = train_pbp_krzy05["situationCode"].str[0].astype(int)
train_pbp_krzy05["home_skaters"] = train_pbp_krzy05["situationCode"].str[1].astype(int)
train_pbp_krzy05["away_goalie"] = train_pbp_krzy05["situationCode"].str[2].astype(int)
train_pbp_krzy05["away_skaters"] = train_pbp_krzy05["situationCode"].str[3].astype(int)
test_pbp_krzy05["home_goalie"] = test_pbp_krzy05["situationCode"].str[0].astype(int)
test_pbp_krzy05["home_skaters"] = test_pbp_krzy05["situationCode"].str[1].astype(int)
test_pbp_krzy05["away_goalie"] = test_pbp_krzy05["situationCode"].str[2].astype(int)
test_pbp_krzy05["away_skaters"] = test_pbp_krzy05["situationCode"].str[3].astype(int)

# Calculate players on ice for each time
train_pbp_krzy05["home_players"] = train_pbp_krzy05["home_goalie"] + train_pbp_krzy05["home_skaters"]
train_pbp_krzy05["away_players"] = train_pbp_krzy05["away_goalie"] + train_pbp_krzy05["away_skaters"]
test_pbp_krzy05["home_players"] = test_pbp_krzy05["home_goalie"] + test_pbp_krzy05["home_skaters"]
test_pbp_krzy05["away_players"] = test_pbp_krzy05["away_goalie"] + test_pbp_krzy05["away_skaters"]

train_pbp_krzy05["Situation"] = train_pbp_krzy05.apply(get_strength, axis = 1) 
test_pbp_krzy05["Situation"] = test_pbp_krzy05.apply(get_strength, axis = 1) 

train_pbp_krzy05.head()

,timeInPeriod,timeRemaining,situationCode,typeDescKey,periodDescriptor.number,details.eventOwnerTeamId,details.xCoord,details.yCoord,details.shotType,gameId,homeTeamDefendingSide,lastEvent,timeSinceLastEvent,id,awayTeam.id,homeTeam.id,abs.xCoord,homeTeamShot,shotSide,adj.xCoord,adj.yCoord,shotDistance,shotDistanceClass,Rebound,home_goalie,home_skaters,away_goalie,away_skaters,home_players,away_players,Situation
0,00:15,19:45,1551,shot-on-goal,1,8,71.0,13.0,slap,2021020003,right,hit,4.0,2021020003,8,10,71.0,False,right,71.0,13.0,22.203603,4,False,1,5,5,1,6,6,EV
1,00:18,19:42,1551,shot-on-goal,1,8,78.0,-22.0,wrist,2021020003,right,missed-shot,0.0,2021020003,8,10,78.0,False,right,78.0,-22.0,24.596748,5,False,1,5,5,1,6,6,EV
2,01:16,18:44,1551,shot-on-goal,1,10,-60.0,-18.0,wrist,2021020003,right,hit,26.0,2021020003,8,10,60.0,True,left,60.0,18.0,34.132096,6,False,1,5,5,1,6,6,EV
3,01:23,18:37,1551,shot-on-goal,1,8,64.0,-16.0,wrist,2021020003,right,shot-on-goal,7.0,2021020003,8,10,64.0,False,right,64.0,-16.0,29.681644,5,False,1,5,5,1,6,6,EV
4,02:50,17:10,1551,shot-on-goal,1,8,31.0,20.0,slap,2021020003,right,blocked-shot,2.0,2021020003,8,10,31.0,False,right,31.0,20.0,61.351447,10,False,1,5,5,1,6,6,EV
